In [2]:
import mmpose
print(mmpose.__version__)


try:
    from mmpose.apis import (get_track_id, inference_top_down_pose_model,
                         init_pose_model, process_mmdet_results,
                         vis_pose_tracking_result, vis_pose_result)
    from mmpose.datasets import DatasetInfo
    has_mmpose = True
except (ImportError, ModuleNotFoundError):
    has_mmpose = False

print(has_mmpose)


0.24.0
apex is not installed
apex is not installed
apex is not installed


/mnt/d/Justine/mmcv/mmcv/cnn/bricks/transformer.py:33: UserWarning: Fail to import ``MultiScaleDeformableAttention`` from ``mmcv.ops.multi_scale_deform_attn``, You should install ``mmcv-full`` if you need this module. 
  warnings.warn('Fail to import ``MultiScaleDeformableAttention`` from '


True


In [3]:

import torch 
use_cuda = torch. cuda. is_available()

print(torch.__version__, torch.cuda.is_available())

1.11.0+cu115 True


In [1]:


import mmdet
print(mmdet.__version__)

try:
    from mmdet.apis import inference_detector, init_detector
    has_mmdet = True
except (ImportError, ModuleNotFoundError):
    has_mmdet = False

print(has_mmdet)

/home/dylanseychell/.pyenv/versions/3.10.0/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.28.2
False


In [3]:

import mmcv
mmcv.collect_env() 

{'sys.platform': 'linux',
 'Python': '3.10.0 (default, Oct 15 2023, 19:20:34) [GCC 10.5.0]',
 'CUDA available': True,
 'GPU 0': 'NVIDIA GeForce GTX 1080 Ti',
 'CUDA_HOME': '/usr',
 'NVCC': 'Build cuda_11.5.r11.5/compiler.30672275_0',
 'GCC': 'gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0',
 'PyTorch': '1.11.0+cu115',
 'PyTorch compiling details': 'PyTorch built with:\n  - GCC 7.3\n  - C++ Version: 201402\n  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications\n  - Intel(R) MKL-DNN v2.5.2 (Git Hash a9302535553c73243c632ad3c4c80beec3d19a1e)\n  - OpenMP 201511 (a.k.a. OpenMP 4.5)\n  - LAPACK is enabled (usually provided by MKL)\n  - NNPACK is enabled\n  - CPU capability usage: AVX2\n  - CUDA Runtime 11.5\n  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_8

In [4]:
import codecs, json, numpy
import numpy as np
import os
import warnings
from argparse import ArgumentParser

import cv2

from mmpose.apis import (get_track_id, inference_top_down_pose_model,
                         init_pose_model, process_mmdet_results,
                         vis_pose_tracking_result, vis_pose_result)
from mmpose.datasets import DatasetInfo

HOME_PATH='/mnt/d/Justine/uni'
def writeJson(val,fname):
  with open(fname, 'w') as data_file:
    json.dump(val, data_file)
    
def write_json(val,fname):
    json_object = json.dumps(val, indent=4)
 
    # Writing to sample.json
    with open(fname, "w") as outfile:
        outfile.write(json_object)
        
try:
    from mmdet.apis import inference_detector, init_detector
    has_mmdet = True
except (ImportError, ModuleNotFoundError):
    has_mmdet = False

det_config = HOME_PATH +'/ViTPose/demo/mmdetection_cfg/faster_rcnn_r50_fpn_coco.py'
det_checkpoint = HOME_PATH + '/ViTPose/configs/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'
pose_config = HOME_PATH +'/ViTPose/configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/coco/ViTPose_huge_coco_256x192.py'#'/ViTPose/configs/wholebody/2d_kpt_sview_rgb_img/topdown_heatmap/coco-wholebody/ViTPose_huge_wholebody_256x192.py'
pose_checkpoint = HOME_PATH +'/ViTPose/configs/wholebody/2d_kpt_sview_rgb_img/topdown_heatmap/coco-wholebody/vitpose+_huge.pth'
#pose_config = HOME_PATH +'/ViTPose/configs/wholebody/2d_kpt_sview_rgb_img/topdown_heatmap/coco-wholebody/hrnet_w48_coco_wholebody_384x288_dark_plus.py'
#pose_checkpoint = HOME_PATH + '/ViTPose/configs/mmpose/top_down/hrnet/hrnet_w48_coco_wholebody_384x288_dark-f5726563_20200918.pth'
pose_config = HOME_PATH +'/ViTPose/configs/wholebody/2d_kpt_sview_rgb_img/topdown_heatmap/coco-wholebody/ViTPose_huge_wholebody_256x192.py'
pose_checkpoint = HOME_PATH + '/ViTPose/configs/wholebody/2d_kpt_sview_rgb_img/topdown_heatmap/coco-wholebody/vitpose+_huge.pth'

out_video_root=HOME_PATH +'/ViTPose/output'
device='cuda:0'
det_cat_id=1
bbox_thr=0.6
kpt_thr=0.3
tracking_thr=0.3
radius=4
thickness=1

det_model = init_detector(
    det_config, det_checkpoint, device=device.lower())
# build the pose model from a config file and a checkpoint file
pose_model = init_pose_model(
    pose_config, pose_checkpoint, device=device.lower())

dataset = pose_model.cfg.data['test']['type']
dataset_info = pose_model.cfg.data['test'].get('dataset_info', None)
if dataset_info is None:
    warnings.warn(
        'Please set `dataset_info` in the config.'
        'Check https://github.com/open-mmlab/mmpose/pull/663 for details.',
        DeprecationWarning)
else:
    dataset_info = DatasetInfo(dataset_info)

def processVideo(video_path):
    cap = cv2.VideoCapture(video_path)
    fps = None

    os.makedirs(out_video_root, exist_ok=True)

    fps = cap.get(cv2.CAP_PROP_FPS)
    size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    videoWriter = cv2.VideoWriter(
        os.path.join(out_video_root,
                     f'{os.path.basename(video_path)}'), fourcc,
        fps, size)

    # optional
    return_heatmap = False

    # e.g. use ('backbone', ) to return backbone feature
    output_layer_names = None

    next_id = 0
    pose_results = []
    all_pose_results = []
    frame_id = 0
    while (cap.isOpened()):
        pose_results_last = pose_results

        flag, img = cap.read()
        if not flag:
            break
        # test a single image, the resulting box is (x1, y1, x2, y2)
        mmdet_results = inference_detector(det_model, img)

        # keep the person class bounding boxes.
        person_results = process_mmdet_results(mmdet_results, det_cat_id)

        # test a single image, with a list of bboxes.
        pose_results, returned_outputs = inference_top_down_pose_model(
            pose_model,
            img,
            person_results,
            bbox_thr=bbox_thr,
            format='xyxy',
            dataset=dataset,
            dataset_info=dataset_info,
            return_heatmap=return_heatmap,
            outputs=output_layer_names)

        # get track id for each person instance
        pose_results, next_id = get_track_id(
            pose_results,
            pose_results_last,
            next_id,
            use_oks=False,
            tracking_thr=tracking_thr,
            use_one_euro=False,
            fps=fps)

        # show the results
        vis_img = vis_pose_tracking_result(
            pose_model,
            img,
            pose_results,
            radius=radius,
            thickness=thickness,
            dataset=dataset,
            dataset_info=dataset_info,
            kpt_score_thr=kpt_thr,
            show=False)

        videoWriter.write(vis_img)
        for pose in pose_results:
            keypoints = []
            for keypoint in pose['keypoints']:
                for point in keypoint:
                    keypoints.append(np.float64(point))
            pose_res = {"image_id": str(frame_id) + ".jpg", "box": pose['bbox'].tolist(), "keypoints": keypoints }
            all_pose_results.append(pose_res)
        
        frame_id += 1
        
    cap.release()
    videoWriter.release()
    write_json((all_pose_results), os.path.join(out_video_root,
                     f'{os.path.basename(video_path)}.json'))


ImportError: cannot import name 'container_abcs' from 'torch._six' (/home/dylanseychell/.pyenv/versions/3.10.0/envs/py310/lib/python3.10/site-packages/torch/_six.py)

In [7]:
directory=HOME_PATH+'/data/videos'
out_video_root=HOME_PATH +'/data/videos/results/test'#ViTPose'

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f) and filename.startswith('World_Athletics_Men_5000m_Oregon_2022_8'):
        print("Processing file: " + f)
        processVideo(f)

Processing file: /mnt/d/Justine/uni/data/videos/World_Athletics_Men_5000m_Oregon_2022_8.mp4


In [14]:

def write_keypoint_results(keypoints, res_file):
    """Write results into a json file."""

    with open(res_file, 'w') as f:
        json.dump(keypoints, f, sort_keys=True, indent=4)

def xywh2cs(image_height, image_width, x, y, w, h):
    """This encodes bbox(x, y, w, h) into (center, scale)
    Args:
        x, y, w, h
    Returns:
        tuple: A tuple containing center and scale.
        - center (np.ndarray[float32](2,)): center of the bbox (x, y).
        - scale (np.ndarray[float32](2,)): scale of the bbox w & h.
    """
    aspect_ratio = image_height / image_width
    center = np.array([x + w * 0.5, y + h * 0.5], dtype=np.float32)
    if w > aspect_ratio * h:
        h = w * 1.0 / aspect_ratio
    elif w < aspect_ratio * h:
        w = h * aspect_ratio

    # pixel std is 200.0
    scale = np.array([w / 200.0, h / 200.0], dtype=np.float32)
    # padding to include proper amount of context
    scale = scale * 1.25
    
    return center, scale

def get_db():
    with open(ann_file) as f:
        anns = json.load(f)

    db = []
    for idx, ann in enumerate(anns['annotations']):
        # get image path
        image_id = ann['image_id']
        image_filename = str().zfill(12) + '.jpg'
        image_file = os.path.join(img_prefix, image_filename)
        # get bbox
        bbox = ann['bbox']
        
        for idx, ann_image in enumerate(anns['images']):
            if ann_image['id'] == image_id:
                image_file = os.path.join(img_prefix, ann_image['file_name'])
                image_height = ann_image['height']
                image_width = ann_image['width']

        center, scale = xywh2cs(image_height, image_width, *bbox)
        # get keypoints
        keypoints = np.array(
            ann['keypoints'], dtype=np.float32).reshape(-1, 3)
        num_joints = keypoints.shape[0]
        joints_3d = np.zeros((num_joints, 3), dtype=np.float32)
        joints_3d[:, :2] = keypoints[:, :2]
        joints_3d_visible = np.zeros((num_joints, 3), dtype=np.float32)
        joints_3d_visible[:, :2] = np.minimum(1, keypoints[:, 2:3])

        sample = {
            'image_file': image_file,
            'center': center,
            'scale': scale,
            'bbox': bbox,
            'rotation': 0,
            'joints_3d': joints_3d,
            'joints_3d_visible': joints_3d_visible,
            'bbox_score': 1,
            'bbox_id': idx,
        }
        db.append(sample)

    return db


def report_metric(res_file, metrics, pck_thr=0.3):
    """Keypoint evaluation.

    Args:
    res_file (str): Json file stored prediction results.
    metrics (str | list[str]): Metric to be performed.
        Options: 'PCK', 'NME'.
    pck_thr (float): PCK threshold, default: 0.3.

    Returns:
    dict: Evaluation results for evaluation metric.
    """
    info_str = []
    db = get_db()

    with open(res_file, 'r') as fin:
        preds = json.load(fin)
    print(len(preds))
    print(len(db))
    assert len(preds) == len(db)

    outputs = []
    gts = []
    masks = []

    for pred, item in zip(preds, db):
        outputs.append(np.array(pred['keypoints'])[:, :-1])
        gts.append(np.array(item['joints_3d'])[:, :-1])
        masks.append((np.array(item['joints_3d_visible'])[:, 0]) > 0)

    outputs = np.array(outputs)
    gts = np.array(gts)
    masks = np.array(masks)

    normalize_factor = get_normalize_factor(gts)

    if 'PCK' in metrics:
        _, pck, _ = keypoint_pck_accuracy(outputs, gts, masks, pck_thr,
                                          normalize_factor)
        info_str.append(('PCK', pck))

    if 'NME' in metrics:
        info_str.append(
            ('NME', keypoint_nme(outputs, gts, masks, normalize_factor)))

    return info_str
    
def sort_and_unique_bboxes(kpts, key='bbox_id'):
    """sort kpts and remove the repeated ones."""
    kpts = sorted(kpts, key=lambda x: x[key])
    num = len(kpts)
    for i in range(num - 1, 0, -1):
        if kpts[i][key] == kpts[i - 1][key]:
            del kpts[i]

    return kpts

def get_normalize_factor(gts):
    """Get inter-ocular distance as the normalize factor, measured as the
    Euclidean distance between the outer corners of the eyes.

    Args:
        gts (np.ndarray[N, K, 2]): Groundtruth keypoint location.

    Return:
        np.ndarray[N, 2]: normalized factor
    """

    interocular = np.linalg.norm(
        gts[:, 0, :] - gts[:, 1, :], axis=1, keepdims=True)
    return np.tile(interocular, [1, 2])

    
def writeJson(val,fname):
  with open(fname, 'w') as data_file:
    json.dump(val, data_file)

In [15]:

from mmpose.apis import (get_track_id, inference_top_down_pose_model,
                         init_pose_model, process_mmdet_results,
                         vis_pose_tracking_result, vis_pose_result)

img_prefix = HOME_PATH+'/data/mpii/images/'
ann_file = HOME_PATH+'/data/COCO-WholeBody/annotations/coco_wholebody_val_v1.0.json'

ann_info = {}
ann_info['flip_pairs'] = [[1, 2], [3, 4], [5, 6], [7, 8], [9, 10],
                               [11, 12], [13, 14], [15, 16]]
ann_info['upper_body_ids'] = (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
ann_info['lower_body_ids'] = (11, 12, 13, 14, 15, 16)

ann_info['joint_weights'] = None
ann_info['use_different_joint_weights'] = False

out_image_root=HOME_PATH +'/data/output/ViTPose'

def processImage(img_root, img):    
    image_name = os.path.join(img_root, img)
    print(image_name)

    det_model = init_detector(
    det_config, det_checkpoint, device=device.lower())
    # test a single image, the resulting box is (x1, y1, x2, y2)
    mmdet_results = inference_detector(det_model, image_name)

    # keep the person class bounding boxes.
    person_results = process_mmdet_results(mmdet_results, det_cat_id)

    # test a single image, with a list of bboxes.

    # optional
    return_heatmap = False

    # e.g. use ('backbone', ) to return backbone feature
    output_layer_names = None

    pose_results, returned_outputs = inference_top_down_pose_model(
        pose_model,
        image_name,
        person_results,
        bbox_thr=bbox_thr,
        format='xyxy',
        dataset=dataset,
        dataset_info=dataset_info,
        return_heatmap=return_heatmap,
        outputs=output_layer_names)

    if out_image_root == '':
        out_file = None
    else:
        os.makedirs(out_image_root, exist_ok=True)
        out_file = os.path.join(out_image_root, f'{img}')

    # show the results
    vis_pose_result(
        pose_model,
        image_name,
        pose_results,
        dataset=dataset,
        dataset_info=dataset_info,
        kpt_score_thr=kpt_thr,
        radius=radius,
        thickness=thickness,
        show=False,
        out_file=out_file)

    kpts = []
    for result in pose_results:
        kpts.append({
            'box': result['bbox'].tolist(),
            'image_id': img,
            'keypoints': result['keypoints'].flatten().tolist()
        })

    res_file = f'{out_file.replace(".jpg","")}.json'
    writeJson(kpts,res_file)
    #metrics = 'PCK'
    #write_keypoint_results(kpts, res_file)
    #info_str = report_metric(res_file, metrics)
    #name_value = OrderedDict(info_str)
    #print(name_value)
    #write_json((name_value), os.path.join(out_image_root,
    #                 f'{os.path.basename(img)}.json'))
    
    

In [8]:
directory=HOME_PATH+'/data/mpii/images'
out_image_root=HOME_PATH +'/data/output/ViTPose/mpii'

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print("Processing file: " + f)
        isExist = os.path.exists(os.path.join(out_image_root, f'{filename}'))
        if not isExist:
            processImage(directory, filename)

Processing file: /mnt/d/Justine/uni/data/mpii/images/000001163.jpg
Processing file: /mnt/d/Justine/uni/data/mpii/images/000003072.jpg
Processing file: /mnt/d/Justine/uni/data/mpii/images/000004812.jpg
Processing file: /mnt/d/Justine/uni/data/mpii/images/000005283.jpg
Processing file: /mnt/d/Justine/uni/data/mpii/images/000013469.jpg
Processing file: /mnt/d/Justine/uni/data/mpii/images/000015774.jpg
Processing file: /mnt/d/Justine/uni/data/mpii/images/000022704.jpg
Processing file: /mnt/d/Justine/uni/data/mpii/images/000024087.jpg
Processing file: /mnt/d/Justine/uni/data/mpii/images/000024100.jpg
Processing file: /mnt/d/Justine/uni/data/mpii/images/000024293.jpg
Processing file: /mnt/d/Justine/uni/data/mpii/images/000025245.jpg
Processing file: /mnt/d/Justine/uni/data/mpii/images/000030708.jpg
Processing file: /mnt/d/Justine/uni/data/mpii/images/000030973.jpg
Processing file: /mnt/d/Justine/uni/data/mpii/images/000033016.jpg
Processing file: /mnt/d/Justine/uni/data/mpii/images/000040154

In [16]:
directory=HOME_PATH+'/data/coco/val2017'

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print("Processing file: " + f)
        processImage(directory, filename)

Processing file: /mnt/d/Justine/uni/data/coco/val2017/000000000139.jpg
/mnt/d/Justine/uni/data/coco/val2017/000000000139.jpg
load checkpoint from local path: /mnt/d/Justine/uni/ViTPose/configs/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
Processing file: /mnt/d/Justine/uni/data/coco/val2017/000000000285.jpg
/mnt/d/Justine/uni/data/coco/val2017/000000000285.jpg
load checkpoint from local path: /mnt/d/Justine/uni/ViTPose/configs/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
Processing file: /mnt/d/Justine/uni/data/coco/val2017/000000000632.jpg
/mnt/d/Justine/uni/data/coco/val2017/000000000632.jpg
load checkpoint from local path: /mnt/d/Justine/uni/ViTPose/configs/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
Processing file: /mnt/d/Justine/uni/data/coco/val2017/000000000724.jpg
/mnt/d/Justine/uni/data/coco/